<a href="https://colab.research.google.com/github/emilienperrin/HACKATON_ELEVEN/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello")

hello


In [ ]:
print("nouvelle cellule")

nouvelle cellule


In [1]:
print("enzo")

enzo
